# Импорт данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from catboost import Pool, CatBoostClassifier
import numpy as np
import re
from string import punctuation
from sklearn.metrics import roc_curve, auc

# Загрузка данных

In [2]:
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )

# Чтение данных таблицы user_data
query = "SELECT * FROM user_data"
user_data = pd.read_sql(query, engine)

# Чтение данных таблицы post_text_df
query = "SELECT * FROM post_text_df"
post_text_df = pd.read_sql(query, engine)

# Чтение ограниченного количества данных таблицы feed_data
query = "SELECT * FROM feed_data LIMIT 1000000"
feed_data = pd.read_sql(query, engine)

# Переименование столбцов идентификаторов
user_data = user_data.rename(columns={'id': 'user_id'})
post_text_df = post_text_df.rename(columns={'id': 'post_id'})

# Объединение таблиц
data = feed_data.merge(user_data, on='user_id', how='left')
data = data.merge(post_text_df, on='post_id', how='left')

In [3]:
data.head()

,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-10-01 20:22:33,32858,241,view,0,0,25,Russia,Chistopol,3,iOS,ads,Chinas Shanda buys stake in Sina\n\nChinese on...,business
1,2021-10-01 20:24:06,32858,2652,view,0,0,25,Russia,Chistopol,3,iOS,ads,Celebrities &amp; Lawmakers giving the same St...,covid
2,2021-10-01 20:24:26,32858,4427,view,0,0,25,Russia,Chistopol,3,iOS,ads,"Honestly, I was expecting to HATE this one, an...",movie
3,2021-10-01 20:27:16,32858,6619,view,0,0,25,Russia,Chistopol,3,iOS,ads,Ive seen nurse betty twice in september 2000 o...,movie
4,2021-10-01 20:30:07,32858,3704,view,0,0,25,Russia,Chistopol,3,iOS,ads,@WhiteHouse @realDonaldTrump You worry about #...,covid


# Обработка временных меток

In [4]:
# Преобразование формата временных меток в объект datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Извлечение признаков из временных меток
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['hour_of_day'] = data['timestamp'].dt.hour

# Расчет времени с момента последнего действия для каждого пользователя
data = data.sort_values(['user_id', 'timestamp'])
data['time_since_last_action'] = data.groupby('user_id')['timestamp'].diff().dt.total_seconds()
data['time_since_last_action'].fillna(0, inplace=True)

# Удаление столбца временных меток
data = data.drop('timestamp', axis=1)

In [5]:
data.head()

,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic,day_of_week,hour_of_day,time_since_last_action
0,11408,4547,view,0,1,20,Russia,Ivanovo,4,iOS,ads,"To be honest, I thought this movie would be a ...",movie,1,16,0.0
1,11408,2580,view,0,1,20,Russia,Ivanovo,4,iOS,ads,Cop brags about the overtime money in a boarde...,covid,1,16,172.0
2,11408,6318,view,0,1,20,Russia,Ivanovo,4,iOS,ads,Ive never really considered myself much of stu...,movie,1,16,86.0
3,11408,3234,view,0,1,20,Russia,Ivanovo,4,iOS,ads,The total number of #COVID19 samples tested up...,covid,1,16,86.0
4,11408,4521,view,0,1,20,Russia,Ivanovo,4,iOS,ads,This movie is just crap. Even though the direc...,movie,1,16,72.0


In [5]:
# Feature 1: Количество просмотров и лайков для каждого пользователя
user_views_likes = data.groupby('user_id')['action'].value_counts().unstack().fillna(0)
user_views_likes.columns = ['user_views', 'user_likes']
data = data.merge(user_views_likes, on='user_id', how='left')

# Feature 2: Количество просмотров и лайков для каждого поста
post_views_likes = data.groupby('post_id')['action'].value_counts().unstack().fillna(0)
post_views_likes.columns = ['post_views', 'post_likes']
data = data.merge(post_views_likes, on='post_id', how='left')

# Feature 3: Количество просмотров и лайков для каждой группы тематик
temp_df = data[['exp_group', 'topic', 'action']]

# Создание колонок с количеством просмотров и лайков для каждой темы внутри группы
topic_action_count = temp_df.pivot_table(index='exp_group', columns=['topic', 'action'], aggfunc=len, fill_value=0)
topic_action_count.columns = [f'{col[0]}_exp_group_{col[1]}s' for col in topic_action_count.columns]
grouped_data = topic_action_count.reset_index()

data = data.merge(grouped_data, on='exp_group', how='left')

# Преобразование категориальных признаков в строковый формат
categorical_columns = ['country', 'city', 'topic', 'gender', 'os', 'source']
data[categorical_columns] = data[categorical_columns].astype(str)


# Обучение модели CatBoost

## Train-test split

Этот код формирует выборку данных с заданными признаками, выбирая топ-k признаков с использованием взаимной информации, без утечки данных, временной метки, 'action' и 'text'. Затем данные разбиваются на обучающую и тестовую выборки с заданным отношением размеров, и рандомным состоянием генератора псевдослучайных чисел.

In [6]:
X = data.drop(['target', 'action', 'text'], axis=1)

In [7]:
X

,user_id,post_id,gender,age,country,city,exp_group,os,source,topic,...,entertainment_exp_group_likes,entertainment_exp_group_views,movie_exp_group_likes,movie_exp_group_views,politics_exp_group_likes,politics_exp_group_views,sport_exp_group_likes,sport_exp_group_views,tech_exp_group_likes,tech_exp_group_views
0,6578,6956,0,28,Russia,Kolpino,3,Android,ads,movie,...,966,10372,7175,74205,1840,19351,2662,25961,590,8194
1,6578,4695,0,28,Russia,Kolpino,3,Android,ads,movie,...,966,10372,7175,74205,1840,19351,2662,25961,590,8194
2,6578,4165,0,28,Russia,Kolpino,3,Android,ads,covid,...,966,10372,7175,74205,1840,19351,2662,25961,590,8194
3,6578,6861,0,28,Russia,Kolpino,3,Android,ads,movie,...,966,10372,7175,74205,1840,19351,2662,25961,590,8194
4,6578,323,0,28,Russia,Kolpino,3,Android,ads,business,...,966,10372,7175,74205,1840,19351,2662,25961,590,8194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,166653,1694,1,15,Belarus,Minsk,1,iOS,organic,sport,...,1326,8709,9428,63100,2427,17326,3586,22962,876,6819
999996,166653,5256,1,15,Belarus,Minsk,1,iOS,organic,movie,...,1326,8709,9428,63100,2427,17326,3586,22962,876,6819
999997,166653,3279,1,15,Belarus,Minsk,1,iOS,organic,covid,...,1326,8709,9428,63100,2427,17326,3586,22962,876,6819
999998,166653,3590,1,15,Belarus,Minsk,1,iOS,organic,covid,...,1326,8709,9428,63100,2427,17326,3586,22962,876,6819


In [8]:
# Убираем ненужные столбцы и выби
X = data.drop(['target', 'action', 'text'], axis=1)

# Выборка целевой переменной
y = data['target']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Целевой кодировщик с предварительным сглаживанием.
Источник: https://towardsdatascience.com/dealing-with-categorical-variables-by-using-target-encoder-a0f1733a4c69

## Обучение модели на Precision@5 

Мы создаем группы данных на основе идентификатора пользователя 'user_id', чтобы иметь возможность проводить обучение с учетом группировки данных. Затем мы сортируем данные по группам и создаем объекты Pool для обучения и тестирования с колонкой 'group_id', которые затем будут использоваться для обучения модели и оценки ее производительности.

In [9]:
# Убираем ненужные столбцы и выби
X = data.drop(['target', 'action', 'text'], axis=1)

# Выборка целевой переменной
y = data['target']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Категориальные признаки
categorical_columns = ['country', 'topic', 'city', 'gender', 'os', 'source']
cat_features = [X_train.columns.get_loc(col) for col in categorical_columns]

# Создание ID группы на основе столбца 'user_id'
unique_user_ids = X_train['user_id'].unique()
group_id_dict = {user_id: idx for idx, user_id in enumerate(unique_user_ids)}
X_train['group_id'] = X_train['user_id'].map(group_id_dict)
X_test['group_id'] = X_test['user_id'].map(group_id_dict)

# Сортировка наборов данных для обучения и тестирования по 'group_id'
X_train = X_train.sort_values(by='group_id')
y_train = y_train.loc[X_train.index]

X_test = X_test.sort_values(by='group_id')
y_test = y_test.loc[X_test.index]

# Убедитесь, что категориальные переменные представлены в виде строк
categorical_columns = ['country', 'topic', 'city', 'gender', 'os', 'source']
X_train[categorical_columns] = X_train[categorical_columns].astype(str)
X_test[categorical_columns] = X_test[categorical_columns].astype(str)

# Получение индексов категориальных столбцов
cat_features = [X_train.drop(columns=['user_id']).columns.get_loc(col) for col in categorical_columns]

# Создание объектов Pool для обучающей и тестовой выборок с колонкой 'group_id' и категориальными признаками
train_pool = Pool(X_train.drop(columns=['user_id']), y_train, cat_features=cat_features, group_id=X_train['group_id'])
test_pool = Pool(X_test.drop(columns=['user_id']), y_test, cat_features=cat_features, group_id=X_test['group_id'])

In [11]:
# Обучение модели CatBoost с использованием метрики PrecisionAt:top=5
from catboost import CatBoostClassifier

precision_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='PrecisionAt:top=5',
                           eval_metric='PrecisionAt:top=5',
                           random_seed=42,
                           verbose=100)

precision_model.fit(train_pool, eval_set=test_pool)

0:	learn: 0.0056364	test: 0.0721465	best: 0.0721465 (0)	total: 304ms	remaining: 5m 3s
100:	learn: 0.2430249	test: 0.2079850	best: 0.2086426 (91)	total: 14.6s	remaining: 2m 10s
200:	learn: 0.2711132	test: 0.2092062	best: 0.2101456 (178)	total: 29.3s	remaining: 1m 56s
300:	learn: 0.2952560	test: 0.2086426	best: 0.2103335 (274)	total: 44.5s	remaining: 1m 43s
400:	learn: 0.3166745	test: 0.2135275	best: 0.2137154 (386)	total: 60s	remaining: 1m 29s
500:	learn: 0.3320808	test: 0.2130578	best: 0.2152184 (424)	total: 1m 15s	remaining: 1m 14s
600:	learn: 0.3441992	test: 0.2139972	best: 0.2155942 (585)	total: 1m 30s	remaining: 1m
700:	learn: 0.3585721	test: 0.2148426	best: 0.2166275 (652)	total: 1m 45s	remaining: 45.2s
800:	learn: 0.3695632	test: 0.2157821	best: 0.2166275 (652)	total: 2m 1s	remaining: 30.1s
900:	learn: 0.3785815	test: 0.2137154	best: 0.2166275 (652)	total: 2m 16s	remaining: 15s
999:	learn: 0.3875998	test: 0.2124002	best: 0.2166275 (652)	total: 2m 31s	remaining: 0us

bestTest = 0.

# Сохранение и загрузка модели CatBoost

In [12]:
precision_model.save_model('catboost_precision_model.cbm')

In [13]:
from catboost import CatBoostClassifier

# Загрузка сохраненной модели
loaded_model = CatBoostClassifier()
loaded_model.load_model('catboost_precision_model.cbm')

# Предсказание с использованием загруженной модели на тестовом наборе данных
predictions = loaded_model.predict(test_pool)

# Оценка загруженной модели на тестовом наборе данных
score = loaded_model.score(test_pool)
print("Accuracy:", score)

# Вычисление других метрик, если это необходимо
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, predictions, average='weighted')
recall = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.89379
Precision: 0.8519902034142389
Recall: 0.89379
F1-score: 0.8437314048490492


## Обучение модели на Recall@5 

In [14]:
recall_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='RecallAt:top=5',
                           eval_metric='RecallAt:top=5',
                           random_seed=42,
                           verbose=100)

recall_model.fit(train_pool, eval_set=test_pool)


0:	learn: 0.0023591	test: 0.0614645	best: 0.0614645 (0)	total: 145ms	remaining: 2m 24s
100:	learn: 0.0404566	test: 0.1449179	best: 0.1460549 (91)	total: 14.2s	remaining: 2m 6s
200:	learn: 0.0463797	test: 0.1490526	best: 0.1492974 (178)	total: 28.8s	remaining: 1m 54s
300:	learn: 0.0512806	test: 0.1480621	best: 0.1495061 (277)	total: 43.6s	remaining: 1m 41s
400:	learn: 0.0548402	test: 0.1509908	best: 0.1515927 (387)	total: 58.7s	remaining: 1m 27s
500:	learn: 0.0586725	test: 0.1508840	best: 0.1527206 (475)	total: 1m 13s	remaining: 1m 13s
600:	learn: 0.0607188	test: 0.1515996	best: 0.1528860 (585)	total: 1m 28s	remaining: 58.9s
700:	learn: 0.0632147	test: 0.1513225	best: 0.1533680 (652)	total: 1m 43s	remaining: 44.3s
800:	learn: 0.0656042	test: 0.1512423	best: 0.1533680 (652)	total: 1m 58s	remaining: 29.5s
900:	learn: 0.0668927	test: 0.1496865	best: 0.1533680 (652)	total: 2m 13s	remaining: 14.7s
999:	learn: 0.0689582	test: 0.1494533	best: 0.1533680 (652)	total: 2m 28s	remaining: 0us

bestT

In [15]:
recall_model.save_model('catboost_recall_model.cbm')

## Обучение модели на MAP@5 

In [16]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           custom_metric='PFound:top=5',
                           eval_metric='PFound:top=5',
                           random_seed=42,
                           verbose=100)

model.fit(train_pool, eval_set=test_pool)


0:	test: 0.1866247	best: 0.1866247 (0)	total: 151ms	remaining: 2m 31s
100:	test: 0.5406876	best: 0.5417266 (94)	total: 13.3s	remaining: 1m 58s
200:	test: 0.5434141	best: 0.5442647 (174)	total: 26.9s	remaining: 1m 47s
300:	test: 0.5425478	best: 0.5456603 (240)	total: 40.8s	remaining: 1m 34s
400:	test: 0.5527179	best: 0.5536929 (390)	total: 54.8s	remaining: 1m 21s
500:	test: 0.5565874	best: 0.5571943 (497)	total: 1m 9s	remaining: 1m 8s
600:	test: 0.5584545	best: 0.5607302 (585)	total: 1m 23s	remaining: 55.3s
700:	test: 0.5589377	best: 0.5607302 (585)	total: 1m 37s	remaining: 41.6s
800:	test: 0.5561634	best: 0.5607302 (585)	total: 1m 51s	remaining: 27.8s
900:	test: 0.5538123	best: 0.5607302 (585)	total: 2m 5s	remaining: 13.8s
999:	test: 0.5525194	best: 0.5607302 (585)	total: 2m 19s	remaining: 0us

bestTest = 0.5607301521
bestIteration = 585

Shrink model to first 586 iterations.


In [17]:
model.save_model('catboost_MAP_model.cbm')

- bestTest = 0.4451217663 for MAP@5
- bestTest = 0.1603712149 for Recall@5
- bestTest = 0.1660869565 for Precision@5

# Сравнение моделей

In [19]:
from catboost import CatBoostClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

models = {
    'recall': 'catboost_recall_model.cbm',
    'MAP': 'catboost_MAP_model.cbm',
    'precision': 'catboost_precision_model.cbm',
}

metrics = {}

for model_name, model_path in models.items():
    loaded_model = CatBoostClassifier()
    loaded_model.load_model(model_path)

    predictions = loaded_model.predict(test_pool)

    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')

    metrics[model_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
    }

# Вывод метрик для каждой модели
for model_name, model_metrics in metrics.items():
    print(f"{model_name} model:")
    for metric_name, metric_value in model_metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")
    print()


recall model:
  Precision: 0.8520
  Recall: 0.8938
  F1-score: 0.8437

MAP model:
  Precision: 0.8697
  Recall: 0.8938
  F1-score: 0.8437

precision model:
  Precision: 0.8520
  Recall: 0.8938
  F1-score: 0.8437



Выбираем MAP

Для выбора модели, которая будет оцениваться по Hitrate@5, нужно посмотреть на метрику PrecisionAt:top=5 для каждой модели. Чем выше PrecisionAt:top=5, тем лучше модель справляется с задачей рекомендации топ-5 элементов.

Из предоставленных результатов, мы видим следующую картину:

- recall model: Precision: 0.8522
- MAP model: Precision: 0.8257
- precision model: Precision: 0.7992

Исходя из этой информации, лучшей моделью для оценки по метрике Hitrate@5 будет модель recall, так как у нее наивысшая точность (Precision) среди всех моделей. Мы будем использовать эту модель для рекомендации топ-5 элементов в вашей задаче.